Install:

In [0]:
pip install git+git://github.com/YunyiShen/camera-trap-classifier.git

Mount google drive, to make sure that the model can read photos from google drive. argument in drive.mount is the mounting point, google drive will become a folder in that directory, just like all Linux systems. You need to input the code generated by google drive. 

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
%cd
%cd "/content/gdrive/My Drive/snapshot_wisconsin_images/"
!pwd

Try the classifier

In [0]:
!ctc.create_dataset_inventory csv -path "./snapshotWI_blank.csv" \
-export_path "./SnapshotWI_models/Data/snapshotWI_blank.json" \
-capture_id_field "id" \
-image_fields path_2 \
-label_fields Blank

In [0]:
!mkdir "./SnapshotWI_models/Data/Blank/"

In [0]:
!ctc.create_dataset -inventory "./SnapshotWI_models/Data/snapshotWI_blank.json" \
-output_dir  "./SnapshotWI_models/Data/Blank/" \
-image_save_side_smallest 400 \
-split_percent 0.9 0.05 0.05 \
-process_images_in_parallel \
-overwrite \
-process_images_in_parallel_size 1000 \
-processes_images_in_parallel_n_processes 8 \
-image_save_quality 75 \
-max_records_per_file 20000

Use the comment below in console to prevent colab time out for up to 12 hrs 

```
function ClickConnect(){console.log("Working");document.querySelector("colab-toolbar-button#connect").click()}setInterval(ClickConnect,60000)```

Now Train the model

In [0]:
!mkdir ./SnapshotWI_models/Models/
!mkdir ./SnapshotWI_models/Models/Blank/
!mkdir ./SnapshotWI_models/Models/Blank/run_Blank/
!mkdir ./SnapshotWI_models/Models/Blank/save_Blank/

In [0]:
!mkdir ./SnapshotWI_models/Models/Blank/run_Blank/12/
!mkdir ./SnapshotWI_models/Models/Blank/save_Blank/12/

In [0]:
!ctc.train \
-train_tfr_path ./SnapshotWI_models/Data/Blank/ \
-val_tfr_path ./SnapshotWI_models/Data/Blank/ \
-test_tfr_path ./SnapshotWI_models/Data/Blank/ \
-class_mapping_json ./SnapshotWI_models/Data/Blank/label_mapping.json \
-run_outputs_dir ./SnapshotWI_models/Models/Blank/run_Blank/12/ \
-model_save_dir ./SnapshotWI_models/Models/Blank/save_Blank/12/ \
-continue_training \
-model_to_load "./SnapshotWI_models/Models/Blank/run_Blank/11/model_best.hdf5" \
-model ResNet18 \
-labels Blank \
-batch_size 128 \
-n_parallel_file_reads 50 \
-buffer_size 512 \
-max_epochs 64 \
-labels_loss_weights 1